# Assignment 4: N-gram Language Models

## ML Pipeline Overview

This assignment demonstrates a complete ML pipeline for **n-gram language modeling with smoothing and interpolation**:

### ML Goal
Build a probabilistic language model that can:
1. **Estimate probabilities** of word sequences using n-grams
2. **Handle unseen n-grams** using smoothing techniques
3. **Evaluate model quality** using perplexity
4. **Combine multiple models** using interpolation

### Training Data
Corpus of sentences used to compute n-gram counts and probabilities

---

### Through-Line Example: "I am happy"

Let's trace how we build a language model for the sentence **"I am happy"**:

#### **Progression from Assignment 1 to Assignment 4**

| Assignment | Task | Method | Example |
|------------|------|--------|----------|
| **A1: Autocomplete** | Complete partial word | Frequency-based prefix matching | "happ" → "happy" (most frequent match) |
| **A3: Spell-check** | Correct misspelled word | Edit distance + frequency | "happpy" → "happy" (1 edit, high frequency) |
| **A4: Language Model** | Predict next word probability | N-gram conditional probability | "I am" → P(happy\|I am) = 0.6 |

**Key Evolution:** 
- A1: "What's the most frequent word with this prefix?"
- A3: "What's the most likely correction given edit distance?"
- A4: **"What's the probability of this word given the previous words?"**

---

### Pipeline Stages

#### **1. N-gram Probability Calculation (Q1)**

**Training Corpus:**
```
I am happy
I am learning
I am happy because I am learning
```

**Build N-gram Counts:**

$$
\begin{align}
C(\text{I am}) &= 3 \\
C(\text{am happy}) &= 2 \\
C(\text{am learning}) &= 2
\end{align}
$$

**Calculate probability using MLE (Maximum Likelihood Estimation):**

$$
\begin{align}
P(\text{happy}|\text{am}) &= \frac{C(\text{am happy})}{C(\text{am})} = \frac{2}{4} = 0.5 \\
P(\text{learning}|\text{am}) &= \frac{C(\text{am learning})}{C(\text{am})} = \frac{2}{4} = 0.5
\end{align}
$$

**Apply Chain Rule for Full Sentence:**

$$
\begin{align}
P(\text{I am happy}) &= P(\text{I}) \times P(\text{am}|\text{I}) \times P(\text{happy}|\text{am}) \\
&= 1.0 \times 1.0 \times 0.5 = 0.5
\end{align}
$$

---

#### **2. Laplace Smoothing (Q2)**

**Problem:** What if we see "am sad" in test set, but it never appeared in training?

$$
P(\text{sad}|\text{am}) = \frac{C(\text{am sad})}{C(\text{am})} = \frac{0}{4} = 0 \quad ❌ \text{ Zero probability!}
$$

**Solution: Add-1 (Laplace) Smoothing**

$$
\begin{align}
P_{\text{laplace}}(\text{sad}|\text{am}) &= \frac{C(\text{am sad}) + 1}{C(\text{am}) + V} \\
&= \frac{0 + 1}{4 + 10000} \quad \text{(V = vocabulary size)} \\
&= 0.0001 \quad ✓ \text{ Small but non-zero!}
\end{align}
$$

**Effect on Known N-grams:**

$$
\begin{align}
P_{\text{laplace}}(\text{happy}|\text{am}) &= \frac{2 + 1}{4 + 10000} = 0.0003
\end{align}
$$

*Note: All probabilities reduced, but unknown gets non-zero value*

---

#### **3. Perplexity Evaluation (Q3)**

**Goal:** Measure how "surprised" the model is by the test set

**Test sentence:** "I am happy"

$$
P(\text{I am happy}) = 0.5
$$

**Perplexity formula (inverse geometric mean of probabilities):**

$$
\text{PP} = P(\text{test\_set})^{-1/M} \quad \text{where } M = \text{number of words}
$$

$$
\text{PP} = 0.5^{-1/3} = 1.26
$$

**Interpretation:**
- **Lower perplexity = better model** (less surprised)
- PP = 1: Perfect prediction (model assigns probability 1)
- PP = 100: Model is as confused as if choosing randomly from 100 words

---

#### **4. Add-k Smoothing (Q4)**

**Problem:** Laplace (k=1) gives too much probability to unseen n-grams

**Solution: Use smaller k (e.g., k=0.1)**

$$
\begin{align}
P_k(\text{sad}|\text{am}) &= \frac{0 + 0.1}{4 + 0.1 \times 10000} = 0.00001 \\
P_k(\text{happy}|\text{am}) &= \frac{2 + 0.1}{4 + 0.1 \times 10000} = 0.00021
\end{align}
$$

*Benefit: Known n-grams keep more of their probability mass*

---

#### **5. Linear Interpolation (Q5)**

**Idea:** Combine trigram, bigram, and unigram probabilities with weights

**Given:** "I am happy"

$$
\begin{align}
P_{\text{trigram}}(\text{happy}|\text{I am}) &= 0.8 \\
P_{\text{bigram}}(\text{happy}|\text{am}) &= 0.5 \\
P_{\text{unigram}}(\text{happy}) &= 0.3
\end{align}
$$

**Weighted combination** $(\lambda_1 + \lambda_2 + \lambda_3 = 1)$:

$$
\begin{align}
P_{\text{interpolated}} &= \lambda_1 \times P_{\text{trigram}} + \lambda_2 \times P_{\text{bigram}} + \lambda_3 \times P_{\text{unigram}} \\
&= 0.7 \times 0.8 + 0.2 \times 0.5 + 0.1 \times 0.3 \\
&= 0.69
\end{align}
$$

**Why This Works:**
- Trigrams: High precision, low coverage (main signal when available)
- Bigrams: Medium precision, medium coverage (helpful fallback)
- Unigrams: Low precision, full coverage (prevents zero probabilities)

---

### Connection to Previous Assignments

| Concept | Assignment 2 (Neural Network) | Assignment 4 (N-grams) |
|---------|-------------------------------|------------------------|
| **Model Type** | Neural (learned weights) | Statistical (count-based) |
| **Training** | Backpropagation + gradient descent | Count n-grams in corpus |
| **Probability** | Sigmoid activation → [0,1] | Conditional probability P(w\|context) |
| **Smoothing** | Regularization, dropout | Laplace, add-k, interpolation |
| **Unknown Handling** | Learn generalizations | Smoothing techniques |
| **Evaluation** | Accuracy, confusion matrix | Perplexity |

**Bridge to Neural LMs:** Later in the course, you'll see how neural networks can learn these probability distributions directly from data, combining the best of both approaches!

---

Let's implement each component!

In [ ]:
import math
import random
import numpy as np
import pandas as pd
from collections import defaultdict, Counter

---

## Q1: N-gram Probability Calculation

### Goal
Calculate the probability of a sentence using n-gram language models and the **chain rule of probability**.

### Mathematical Foundation

#### **Chain Rule of Probability**
Any joint probability can be decomposed:

$$
P(w_1, w_2, w_3, \ldots, w_n) = P(w_1) \times P(w_2|w_1) \times P(w_3|w_1,w_2) \times \cdots \times P(w_n|w_1\ldots w_{n-1})
$$

#### **Markov Assumption (N-gram approximation)**
Assume each word depends only on the previous (n-1) words:

**Bigram (n=2):**

$$
P(w_1, w_2, w_3) \approx P(w_1) \times P(w_2|w_1) \times P(w_3|w_2)
$$

**Trigram (n=3):**

$$
P(w_1, w_2, w_3, w_4) \approx P(w_1, w_2) \times P(w_3|w_1,w_2) \times P(w_4|w_2,w_3)
$$

#### **Maximum Likelihood Estimation (MLE)**
Estimate probabilities from counts:

$$
P(w_n|w_1\ldots w_{n-1}) = \frac{C(w_1\ldots w_n)}{C(w_1\ldots w_{n-1})}
$$

### Example

**Training Corpus:**
```
<s> I am happy </s>
<s> I am learning </s>
```

**Bigram Counts:**

$$
\begin{align}
C(\text{<s> I}) &= 2 \\
C(\text{I am}) &= 2 \\
C(\text{am happy}) &= 1 \\
C(\text{am learning}) &= 1
\end{align}
$$

**Calculate** $P(\text{<s> I am happy </s>})$:

$$
\begin{align}
P(\text{I}|\text{<s>}) &= \frac{C(\text{<s> I})}{C(\text{<s>})} = \frac{2}{2} = 1.0 \\
P(\text{am}|\text{I}) &= \frac{C(\text{I am})}{C(\text{I})} = \frac{2}{2} = 1.0 \\
P(\text{happy}|\text{am}) &= \frac{C(\text{am happy})}{C(\text{am})} = \frac{1}{2} = 0.5 \\
P(\text{</s>}|\text{happy}) &= \frac{C(\text{happy </s>})}{C(\text{happy})} = \frac{1}{1} = 1.0 \\
\\
P(\text{sentence}) &= 1.0 \times 1.0 \times 0.5 \times 1.0 = 0.5
\end{align}
$$

### Tips
- Build count dictionaries for n-grams and (n-1)-grams
- Use `defaultdict(int)` to handle missing keys gracefully
- For trigrams, condition on previous 2 words: $P(w_3|w_1,w_2)$
- Remember to include `<s>` and `</s>` tokens!

In [ ]:
# Sample data for testing
training_corpus = [
    ['<s>', 'I', 'am', 'happy', '</s>'],
    ['<s>', 'I', 'am', 'learning', '</s>'],
    ['<s>', 'I', 'am', 'happy', 'because', 'I', 'am', 'learning', '</s>']
]

def build_ngram_counts(corpus, n):
    """
    Build n-gram count dictionary from corpus.
    
    Args:
        corpus: List of tokenized sentences
        n: N-gram order (2 for bigram, 3 for trigram)
    
    Returns:
        Dictionary mapping n-grams (as tuples) to counts
    """
    counts = defaultdict(int)
    
    for sentence in corpus:
        # Slide window of size n over sentence
        for i in range(len(sentence) - n + 1):
            ngram = tuple(sentence[i:i+n])
            counts[ngram] += 1
    
    return counts

# Test the function
bigram_counts = build_ngram_counts(training_corpus, 2)
print("Bigram counts:")
for bigram, count in sorted(bigram_counts.items()):
    print(f"  {bigram}: {count}")

In [ ]:
#@title Q1: N-gram Probability

def calculate_ngram_probability(sentence, ngram_counts, n):
    """
    Calculate probability of a sentence using n-gram model.
    
    Args:
        sentence: List of tokens (should include <s> and </s>)
        ngram_counts: Dictionary of n-gram counts
        n: Order of n-gram (2 for bigram, 3 for trigram)
    
    Returns:
        Probability of the sentence
    """
    # Build (n-1)-gram counts for denominator
    prefix_counts = defaultdict(int)
    for ngram in ngram_counts:
        prefix = ngram[:-1]  # All but last word
        prefix_counts[prefix] += ngram_counts[ngram]
    
    probability = 1.0
    
    # Calculate probability using chain rule
    for i in range(len(sentence) - n + 1):
        ngram = tuple(sentence[i:i+n])
        prefix = ngram[:-1]
        
        # P(wₙ|w₁...wₙ₋₁) = C(w₁...wₙ) / C(w₁...wₙ₋₁)
        if prefix_counts[prefix] > 0:
            prob = ngram_counts[ngram] / prefix_counts[prefix]
        else:
            prob = 0  # Unseen context
        
        probability *= prob
    
    return probability

# Test with through-line example
test_sentence = ['<s>', 'I', 'am', 'happy', '</s>']
prob = calculate_ngram_probability(test_sentence, bigram_counts, 2)
print(f"\nP({' '.join(test_sentence)}) = {prob}")
print(f"This means: The model assigns {prob*100:.1f}% probability to this sentence")

---

## Q2: Laplace (Add-1) Smoothing

### The Zero Probability Problem

**Scenario:** Training corpus has never seen "am sad"

$$
P(\text{sad}|\text{am}) = \frac{C(\text{am sad})}{C(\text{am})} = \frac{0}{4} = 0
$$

**Consequence:** Entire sentence probability becomes 0!

$$
\begin{align}
P(\text{I am sad}) &= P(\text{I}) \times P(\text{am}|\text{I}) \times P(\text{sad}|\text{am}) \\
&= 1.0 \times 1.0 \times 0 = 0 \quad ❌
\end{align}
$$

### Laplace Smoothing Solution

**Idea:** Pretend we saw every possible n-gram at least once

**Formula:**

$$
P_{\text{laplace}}(w_n|w_1\ldots w_{n-1}) = \frac{C(w_1\ldots w_n) + 1}{C(w_1\ldots w_{n-1}) + V}
$$

where $V$ = vocabulary size (number of unique words)

### Visual Intuition: Redistributing Probability Mass

**Before Smoothing (Bigrams after "am"):**
```
am happy:   ████████████ 50%
am learning:████████████ 50%
am sad:                   0%  ← Problem!
am excited:               0%
... (9,996 other words)   0%
```

**After Laplace Smoothing (V=10,000):**
```
am happy:   ██ 0.03%  (was 50%)
am learning:██ 0.03%  (was 50%)
am sad:     █  0.01%  (was 0%)  ✓
am excited: █  0.01%  (was 0%)  ✓
... each of 10,000 words gets 0.01%
```

### Trade-off
✅ **Benefit:** No more zero probabilities  
❌ **Cost:** Known n-grams lose most of their probability (50% → 0.03%)

### Example Calculation

**Original:**

$$
C(\text{am happy}) = 2, \quad C(\text{am}) = 4
$$

$$
P(\text{happy}|\text{am}) = \frac{2}{4} = 0.5
$$

**With Laplace (V = 10,000):**

$$
P_{\text{laplace}}(\text{happy}|\text{am}) = \frac{2 + 1}{4 + 10000} = \frac{3}{10004} \approx 0.0003
$$

**Unseen n-gram:**

$$
P_{\text{laplace}}(\text{sad}|\text{am}) = \frac{0 + 1}{4 + 10000} = \frac{1}{10004} \approx 0.0001
$$

### Tips
- Always add V to denominator (not just V×n)
- V should be vocabulary size, not number of n-grams
- This method works for any n-gram order

In [ ]:
#@title Q2: Laplace Smoothing

def calculate_laplace_probability(ngram, ngram_counts, vocab_size):
    """
    Calculate n-gram probability with Laplace (add-1) smoothing.
    
    Args:
        ngram: Tuple of words (e.g., ('am', 'happy'))
        ngram_counts: Dictionary of n-gram counts
        vocab_size: Number of unique words in vocabulary
    
    Returns:
        Smoothed probability
    """
    # Get n-gram count (0 if unseen)
    ngram_count = ngram_counts.get(ngram, 0)
    
    # Get prefix count (all but last word)
    prefix = ngram[:-1]
    prefix_count = sum(count for ng, count in ngram_counts.items() 
                      if ng[:-1] == prefix)
    
    # Laplace formula: (C + 1) / (C_prefix + V)
    probability = (ngram_count + 1) / (prefix_count + vocab_size)
    
    return probability

# Build vocabulary from training corpus
vocabulary = set()
for sentence in training_corpus:
    vocabulary.update(sentence)
V = len(vocabulary)

print(f"Vocabulary size: {V}\n")

# Test with known bigram
known_bigram = ('am', 'happy')
prob_known = calculate_laplace_probability(known_bigram, bigram_counts, V)
print(f"P_laplace{known_bigram} = {prob_known:.4f}")

# Test with unseen bigram
unseen_bigram = ('am', 'sad')
prob_unseen = calculate_laplace_probability(unseen_bigram, bigram_counts, V)
print(f"P_laplace{unseen_bigram} = {prob_unseen:.4f}")

print(f"\nNotice: Unseen n-gram gets non-zero probability!")

---

## Q3: Perplexity

### What is Perplexity?

**Intuition:** Perplexity measures how "confused" or "surprised" your language model is by the test data.

**Lower perplexity = Better model**

### Mathematical Definition

Perplexity is the **inverse probability** of the test set, normalized by the number of words:

$$
\text{PP}(W) = P(w_1, w_2, \ldots, w_n)^{-1/N}
$$

where $N$ is the number of words in test set.

### Why Use Perplexity Instead of Probability?

**Problem with raw probability:**

$$
\begin{align}
P(\text{"I am happy"}) &= 0.5 \quad \text{(3 words)} \\
P(\text{"I am happy because I am learning"}) &= 0.01 \quad \text{(7 words)}
\end{align}
$$

*Longer sentences always have lower probability! Can't compare.*

**Solution with perplexity:**

$$
\begin{align}
\text{PP}(\text{"I am happy"}) &= 0.5^{-1/3} = 1.26 \\
\text{PP}(\text{"I am happy because I am learning"}) &= 0.01^{-1/7} = 1.86
\end{align}
$$

*Now we can compare! First sentence has lower perplexity → better modeled*

### Interpretation Guide

| Perplexity | Meaning | Example |
|------------|---------|----------|
| **1** | Perfect prediction | Model assigns P=1 to every word |
| **10** | Choosing from ~10 words | Model is uncertain among 10 likely words |
| **100** | Choosing from ~100 words | Model is confused, like random guessing from 100 options |
| **1000+** | Very confused | Poor model, almost random |

### Step-by-Step Example

**Test sentence:** "I am happy"  
**Bigram probabilities:** $P(\text{I}|\text{<s>})=1.0$, $P(\text{am}|\text{I})=1.0$, $P(\text{happy}|\text{am})=0.5$, $P(\text{</s>}|\text{happy})=1.0$

**Step 1: Calculate sentence probability**

$$
P(\text{sentence}) = 1.0 \times 1.0 \times 0.5 \times 1.0 = 0.5
$$

**Step 2: Count words (excluding `<s>`, `</s>`)**

$$
N = 3 \quad \text{[I, am, happy]}
$$

**Step 3: Calculate perplexity**

$$
\text{PP} = 0.5^{-1/3} = \frac{1}{0.5^{1/3}} = \frac{1}{0.794} = 1.26
$$

**Interpretation:** The model is about as confused as if it were choosing from 1.26 equally likely words. This is very good!

### Connection to Cross-Entropy

Perplexity is $2^{\text{cross-entropy}}$:

$$
\begin{align}
\text{Cross-Entropy} &= -\frac{1}{N} \sum \log_2 P(w_i|\text{context}) \\
\text{Perplexity} &= 2^{\text{Cross-Entropy}}
\end{align}
$$

This connects to information theory: perplexity measures average branching factor.

### Tips
- Use `math.pow()` for fractional exponents
- Typically count only content words (exclude `<s>` and `</s>`)
- Lower perplexity = better language model
- Compare perplexity only on the same test set

In [ ]:
#@title Q3: Perplexity

def calculate_perplexity(test_sentences, ngram_counts, n):
    """
    Calculate perplexity of test set given n-gram model.
    
    Args:
        test_sentences: List of tokenized test sentences
        ngram_counts: Dictionary of n-gram counts from training
        n: Order of n-gram
    
    Returns:
        Perplexity value
    """
    total_log_prob = 0
    total_words = 0
    
    # Build prefix counts
    prefix_counts = defaultdict(int)
    for ngram in ngram_counts:
        prefix = ngram[:-1]
        prefix_counts[prefix] += ngram_counts[ngram]
    
    for sentence in test_sentences:
        # Count words (excluding <s> and </s>)
        words = [w for w in sentence if w not in ['<s>', '</s>']]
        total_words += len(words)
        
        # Calculate log probability of sentence
        for i in range(len(sentence) - n + 1):
            ngram = tuple(sentence[i:i+n])
            prefix = ngram[:-1]
            
            if prefix_counts[prefix] > 0:
                prob = ngram_counts[ngram] / prefix_counts[prefix]
            else:
                prob = 1e-10  # Small probability for unseen
            
            # Avoid log(0)
            if prob > 0:
                total_log_prob += math.log(prob)
            else:
                total_log_prob += math.log(1e-10)
    
    # Perplexity = P(test)^(-1/N)
    avg_log_prob = total_log_prob / total_words
    perplexity = math.exp(-avg_log_prob)
    
    return perplexity

# Test on training data (should be low perplexity)
test_set = [['<s>', 'I', 'am', 'happy', '</s>']]
pp = calculate_perplexity(test_set, bigram_counts, 2)
print(f"Perplexity on 'I am happy': {pp:.2f}")
print(f"Interpretation: Model is as confused as choosing from {pp:.1f} equally likely words")

# Test on unseen data
unseen_test = [['<s>', 'I', 'am', 'sad', '</s>']]
pp_unseen = calculate_perplexity(unseen_test, bigram_counts, 2)
print(f"\nPerplexity on 'I am sad': {pp_unseen:.2f}")
print(f"Higher perplexity indicates unseen n-gram!")

---

## Q4: Add-k Smoothing

### The Laplace Problem

Laplace (add-1) smoothing is too aggressive:

$$
\begin{align}
\text{With } V &= 10,000: \\
P_{\text{laplace}}(\text{happy}|\text{am}) &= \frac{2 + 1}{4 + 10000} = 0.0003 \\
P_{\text{laplace}}(\text{sad}|\text{am}) &= \frac{0 + 1}{4 + 10000} = 0.0001
\end{align}
$$

*Ratio is only 3:1, but original was infinite!  
Unseen n-grams get too much probability*

### Add-k Smoothing Solution

**Idea:** Add a smaller value $k$ instead of always adding 1

**Formula:**

$$
P_k(w_n|w_1\ldots w_{n-1}) = \frac{C(w_1\ldots w_n) + k}{C(w_1\ldots w_{n-1}) + k \times V}
$$

### Comparing Different k Values

Given: $C(\text{am happy}) = 2$, $C(\text{am}) = 4$, $V = 10,000$

| k value | $P(\text{happy}|\text{am})$ | $P(\text{sad}|\text{am})$ | Ratio | Comment |
|---------|--------------|------------|-------|----------|
| **k = 0** | $2/4 = 0.5$ | $0/4 = 0$ | ∞ | No smoothing, zero prob problem |
| **k = 0.01** | $2.01/4.1 = 0.490$ | $0.01/4.1 = 0.0024$ | 200:1 | Gentle smoothing |
| **k = 0.1** | $2.1/14 = 0.150$ | $0.1/14 = 0.0071$ | 21:1 | Light smoothing |
| **k = 1** | $3/10004 = 0.0003$ | $1/10004 = 0.0001$ | 3:1 | Laplace (too aggressive) |

### Choosing k

**Rule of thumb:**
- $k \in [0.01, 0.1]$ for most tasks
- Smaller $k$ = preserve original probabilities more
- Larger $k$ = redistribute more probability to unseen

**Selection method:**
1. Try different $k$ values on validation set
2. Choose $k$ that gives lowest perplexity
3. Different $k$ works better for different corpus sizes

### Visual Comparison

**k = 0 (No smoothing):**
```
am happy:   ████████████████████████ 50%
am learning:████████████████████████ 50%
am sad:                               0%  ❌
```

**k = 0.1 (Add-k):**
```
am happy:   ████████████████ 15%
am learning:████████████████ 15%
am sad:     █ 0.7%  ✓ (small but non-zero)
```

**k = 1 (Laplace):**
```
am happy:   █ 0.03%
am learning:█ 0.03%
am sad:     █ 0.01%
# All probabilities too small!
```

### Tips
- Add $k$ to numerator, $k \times V$ to denominator (not just $V$)
- When $k=1$, this becomes Laplace smoothing
- Smaller $k$ usually works better for large vocabularies
- Use validation set to tune $k$

In [ ]:
#@title Q4: Add-k Smoothing

def calculate_add_k_probability(ngram, ngram_counts, vocab_size, k):
    """
    Calculate n-gram probability with add-k smoothing.
    
    Args:
        ngram: Tuple of words
        ngram_counts: Dictionary of n-gram counts
        vocab_size: Number of unique words
        k: Smoothing parameter (0 < k ≤ 1)
    
    Returns:
        Smoothed probability
    """
    # Get n-gram count
    ngram_count = ngram_counts.get(ngram, 0)
    
    # Get prefix count
    prefix = ngram[:-1]
    prefix_count = sum(count for ng, count in ngram_counts.items() 
                      if ng[:-1] == prefix)
    
    # Add-k formula: (C + k) / (C_prefix + k×V)
    probability = (ngram_count + k) / (prefix_count + k * vocab_size)
    
    return probability

# Compare different k values
k_values = [0, 0.01, 0.1, 0.5, 1.0]
test_bigrams = [('am', 'happy'), ('am', 'sad')]  # Known and unknown

print("Comparing different k values:\n")
print(f"{'k':<6} | {'P(happy|am)':<12} | {'P(sad|am)':<12} | Ratio")
print("-" * 50)

for k in k_values:
    if k == 0:
        # No smoothing - manual calculation
        p_happy = 2/4  # C(am happy) / C(am)
        p_sad = 0
        ratio = "∞" if p_sad == 0 else f"{p_happy/p_sad:.1f}:1"
    else:
        p_happy = calculate_add_k_probability(test_bigrams[0], bigram_counts, V, k)
        p_sad = calculate_add_k_probability(test_bigrams[1], bigram_counts, V, k)
        ratio = f"{p_happy/p_sad:.1f}:1" if p_sad > 0 else "∞"
    
    print(f"{k:<6} | {p_happy:<12.6f} | {p_sad:<12.6f} | {ratio}")

print("\nObservation: Smaller k preserves the probability ratio better!")

---

## Q5: Linear Interpolation

### The Smoothing Hierarchy Problem

Different n-gram orders have different strengths:

| N-gram | Strength | Weakness | Example |
|--------|----------|----------|----------|
| **Trigram** | High precision (specific context) | Low coverage (sparse) | $P(\text{happy}|\text{I am}) = 0.8$ (but many unseen trigrams) |
| **Bigram** | Medium precision | Medium coverage | $P(\text{happy}|\text{am}) = 0.5$ (more data, less specific) |
| **Unigram** | Always available | No context | $P(\text{happy}) = 0.3$ (just word frequency) |

### Interpolation Solution

**Idea:** Combine all n-gram orders with learned weights!

**Formula:**

$$
\begin{align}
P_{\text{interp}}(w_3|w_1,w_2) &= \lambda_1 \times P(w_3|w_1,w_2) \quad \text{# Trigram} \\
&+ \lambda_2 \times P(w_3|w_2) \quad \text{# Bigram} \\
&+ \lambda_3 \times P(w_3) \quad \text{# Unigram}
\end{align}
$$

where $\lambda_1 + \lambda_2 + \lambda_3 = 1$ (weights sum to 1)

### Why This Works

**Case 1: Seen trigram**

$$
\begin{align}
\text{"I am happy"} &\text{ appears often in training} \\
P_{\text{trigram}}(\text{happy}|\text{I,am}) &= 0.8 \quad \text{(High confidence)} \\
P_{\text{bigram}}(\text{happy}|\text{am}) &= 0.5 \quad \text{(Medium)} \\
P_{\text{unigram}}(\text{happy}) &= 0.3 \quad \text{(Low)}
\end{align}
$$

$$
\begin{align}
\text{Typical weights: } &\lambda_1=0.7, \lambda_2=0.2, \lambda_3=0.1 \\
P_{\text{interp}} &= 0.7 \times 0.8 + 0.2 \times 0.5 + 0.1 \times 0.3 = 0.69
\end{align}
$$

*Trigram dominates ✓*

**Case 2: Unseen trigram, seen bigram**

$$
\begin{align}
\text{"I am excited"} &\text{ never seen, but "am excited" exists} \\
P_{\text{trigram}}(\text{excited}|\text{I,am}) &= 0.0 \quad \text{(No data)} \\
P_{\text{bigram}}(\text{excited}|\text{am}) &= 0.4 \quad \text{(Has data!)} \\
P_{\text{unigram}}(\text{excited}) &= 0.2 \quad \text{(Background)}
\end{align}
$$

$$
P_{\text{interp}} = 0.7 \times 0.0 + 0.2 \times 0.4 + 0.1 \times 0.2 = 0.10
$$

*Bigram saves us! ✓*

**Case 3: Completely unseen words**

$$
\begin{align}
\text{"I am discombobulated"} &\text{ - never seen anything} \\
P_{\text{trigram}}(\text{discombobulated}|\text{I,am}) &= 0.0 \\
P_{\text{bigram}}(\text{discombobulated}|\text{am}) &= 0.0 \\
P_{\text{unigram}}(\text{discombobulated}) &= 0.0001 \quad \text{(Rare word)}
\end{align}
$$

$$
P_{\text{interp}} = 0.7 \times 0.0 + 0.2 \times 0.0 + 0.1 \times 0.0001 = 0.00001
$$

*Unigram provides fallback ✓*

### Learning Lambda Weights

**Method: Optimize on held-out validation set**

```python
# 1. Split data: 80% train, 10% validation, 10% test
# 2. Build n-gram models on training set
# 3. Try different λ combinations on validation set:

for λ₁ in [0.5, 0.6, 0.7, 0.8]:
    for λ₂ in [0.1, 0.2, 0.3]:
        λ₃ = 1 - λ₁ - λ₂  # Ensure sum = 1
        perplexity = evaluate_on_validation(λ₁, λ₂, λ₃)
        
# 4. Choose λ values with lowest perplexity
# Typical result: λ₁ ≈ 0.7, λ₂ ≈ 0.2, λ₃ ≈ 0.1
```

### Comparison with Other Methods

| Method | Unseen Trigram | Seen Bigram | Seen Unigram | Result |
|--------|----------------|-------------|--------------|--------|
| **No smoothing** | 0 | 0.4 | 0.2 | **0** (fails!) |
| **Laplace k=1** | 0.0001 | 0.0004 | 0.002 | 0.0001 (all too small) |
| **Add-k (k=0.1)** | 0.001 | 0.08 | 0.15 | 0.001 (better) |
| **Back-off** | Uses bigram | 0.4 | 0.2 | 0.4 (uses bigram only) |
| **Interpolation** | Blends all | 0.4 | 0.2 | **0.10** (combines strengths!) |

### Tips
- Weights must sum to 1: $\lambda_1 + \lambda_2 + \lambda_3 = 1$
- Higher-order n-grams usually get larger weights ($\lambda_1 > \lambda_2 > \lambda_3$)
- Can combine with smoothing: interpolate smoothed probabilities
- Use EM algorithm for more sophisticated weight learning

In [ ]:
#@title Q5: Linear Interpolation

def calculate_interpolated_probability(trigram, unigram_counts, bigram_counts, 
                                       trigram_counts, lambda_1, lambda_2, lambda_3):
    """
    Calculate probability using linear interpolation of trigram, bigram, unigram.
    
    Args:
        trigram: Tuple of 3 words (w1, w2, w3)
        unigram_counts: Dictionary of word counts
        bigram_counts: Dictionary of bigram counts
        trigram_counts: Dictionary of trigram counts
        lambda_1, lambda_2, lambda_3: Weights (must sum to 1)
    
    Returns:
        Interpolated probability
    """
    w1, w2, w3 = trigram
    
    # Calculate trigram probability P(w3|w1,w2)
    trigram_prefix = (w1, w2)
    trigram_prefix_count = sum(c for ng, c in trigram_counts.items() 
                               if ng[:2] == trigram_prefix)
    if trigram_prefix_count > 0:
        p_trigram = trigram_counts.get(trigram, 0) / trigram_prefix_count
    else:
        p_trigram = 0
    
    # Calculate bigram probability P(w3|w2)
    bigram = (w2, w3)
    bigram_prefix = (w2,)
    bigram_prefix_count = sum(c for ng, c in bigram_counts.items() 
                             if ng[:1] == bigram_prefix)
    if bigram_prefix_count > 0:
        p_bigram = bigram_counts.get(bigram, 0) / bigram_prefix_count
    else:
        p_bigram = 0
    
    # Calculate unigram probability P(w3)
    total_words = sum(unigram_counts.values())
    p_unigram = unigram_counts.get(w3, 0) / total_words
    
    # Linear interpolation
    p_interpolated = (lambda_1 * p_trigram + 
                     lambda_2 * p_bigram + 
                     lambda_3 * p_unigram)
    
    return p_interpolated, p_trigram, p_bigram, p_unigram

# Build unigram and trigram counts from our corpus
unigram_counts = build_ngram_counts(training_corpus, 1)
# Convert tuples to single words for unigrams
unigram_counts = {k[0]: v for k, v in unigram_counts.items()}

trigram_counts = build_ngram_counts(training_corpus, 3)

# Test with typical weights
lambda_1, lambda_2, lambda_3 = 0.7, 0.2, 0.1

print(f"Lambda weights: λ₁={lambda_1}, λ₂={lambda_2}, λ₃={lambda_3}")
print(f"(Weights sum to {lambda_1 + lambda_2 + lambda_3})\n")

# Test Case 1: Seen trigram
test_trigram = ('I', 'am', 'happy')
p_int, p_tri, p_bi, p_uni = calculate_interpolated_probability(
    test_trigram, unigram_counts, bigram_counts, trigram_counts,
    lambda_1, lambda_2, lambda_3
)

print(f"Test: {test_trigram}")
print(f"  P_trigram(happy|I,am) = {p_tri:.3f}")
print(f"  P_bigram(happy|am)    = {p_bi:.3f}")
print(f"  P_unigram(happy)      = {p_uni:.3f}")
print(f"  P_interpolated        = {p_int:.3f}")
print(f"  Calculation: {lambda_1}×{p_tri:.3f} + {lambda_2}×{p_bi:.3f} + {lambda_3}×{p_uni:.3f} = {p_int:.3f}")

# Test Case 2: Unseen trigram (but seen bigram)
print(f"\n" + "="*50)
test_trigram2 = ('am', 'learning', '</s>')
p_int2, p_tri2, p_bi2, p_uni2 = calculate_interpolated_probability(
    test_trigram2, unigram_counts, bigram_counts, trigram_counts,
    lambda_1, lambda_2, lambda_3
)

print(f"Test: {test_trigram2}")
print(f"  P_trigram(</s>|am,learning) = {p_tri2:.3f}")
print(f"  P_bigram(</s>|learning)     = {p_bi2:.3f}")
print(f"  P_unigram(</s>)             = {p_uni2:.3f}")
print(f"  P_interpolated              = {p_int2:.3f}")
print(f"  Note: Bigram and unigram compensate for unseen trigram!")

---

## Putting It All Together: Complete Language Model Pipeline

Now let's demonstrate the complete pipeline with all techniques:

In [ ]:
print("=" * 70)
print("COMPLETE N-GRAM LANGUAGE MODEL PIPELINE")
print("=" * 70)

print("\n1. TRAINING CORPUS:")
for i, sent in enumerate(training_corpus, 1):
    print(f"   {i}. {' '.join(sent)}")

print("\n2. N-GRAM COUNTS:")
print(f"   Vocabulary size: {V}")
print(f"   Unique bigrams: {len(bigram_counts)}")
print(f"   Unique trigrams: {len(trigram_counts)}")

print("\n3. PROBABILITY METHODS COMPARISON:")
test_sent = ['<s>', 'I', 'am', 'happy', '</s>']
print(f"   Test sentence: {' '.join(test_sent)}\n")

# Method 1: Plain MLE
prob_mle = calculate_ngram_probability(test_sent, bigram_counts, 2)
print(f"   Plain MLE:              P = {prob_mle:.4f}")

# Method 2: Laplace smoothing  
prob_laplace = 1.0
for i in range(len(test_sent) - 1):
    bg = tuple(test_sent[i:i+2])
    prob_laplace *= calculate_laplace_probability(bg, bigram_counts, V)
print(f"   Laplace (k=1):          P = {prob_laplace:.4f}")

# Method 3: Add-k smoothing
prob_add_k = 1.0
k = 0.1
for i in range(len(test_sent) - 1):
    bg = tuple(test_sent[i:i+2])
    prob_add_k *= calculate_add_k_probability(bg, bigram_counts, V, k)
print(f"   Add-k (k=0.1):          P = {prob_add_k:.4f}")

print("\n4. PERPLEXITY EVALUATION:")
pp_mle = calculate_perplexity([test_sent], bigram_counts, 2)
print(f"   Plain MLE perplexity:   {pp_mle:.2f}")
print(f"   Interpretation: Model choosing from ~{pp_mle:.0f} equally likely words")

print("\n5. INTERPOLATION (for trigrams):")
# Build sentence with trigram context
trigram_example = ('I', 'am', 'happy')
p_int, p_tri, p_bi, p_uni = calculate_interpolated_probability(
    trigram_example, unigram_counts, bigram_counts, trigram_counts,
    0.7, 0.2, 0.1
)
print(f"   P_interp(happy|I,am) = {p_int:.4f}")
print(f"   (Combined: {0.7}×{p_tri:.2f} + {0.2}×{p_bi:.2f} + {0.1}×{p_uni:.2f})")

print("\n" + "=" * 70)
print("SUMMARY: Choose method based on your needs:")
print("  - Plain MLE: Fast, but fails on unseen n-grams")
print("  - Laplace: Simple, but too aggressive")
print("  - Add-k: Better balance, tune k on validation set")
print("  - Interpolation: Best performance, combines all orders")
print("=" * 70)

---

## Next Steps: From N-grams to Neural Language Models

### Limitations of N-gram Models

1. **Sparsity:** Most possible n-grams never appear in training
2. **Fixed context:** Can't capture dependencies beyond n-1 words
3. **No generalization:** "cat sat" and "dog sat" treated as completely different
4. **Storage:** Need to store counts for millions of n-grams

### Neural Language Models (Coming Soon!)

**Key improvements:**
- **Learned representations:** Words with similar meanings get similar vectors
- **Generalization:** If model knows "cat sat", it can generalize to "dog sat"
- **Variable context:** RNNs/Transformers can use arbitrarily long context
- **Compact:** Store learned weights instead of all n-gram counts

**But n-grams are still valuable:**
- Baseline for comparison
- Interpretable (can inspect exact counts)
- No training required (just count!)
- Still used in hybrid models

You're now ready to understand how modern language models like GPT build on these foundations!